In [10]:
import os
from dotenv import load_dotenv

load_dotenv()

goodfire_api_key = os.getenv('GOODFIRE_API_KEY')
goodfire_api_key

'sk-goodfire-oOExgMvlySAV4tL_B5F1l18ADVdzNunC7XP2_6Rinvz72Y8hKVh6GA'

In [11]:
import goodfire

client = goodfire.Client(
    goodfire_api_key
  )

# Instantiate a model variant
variant = goodfire.Variant("meta-llama/Meta-Llama-3-8B-Instruct")

In [12]:
for token in client.chat.completions.create(
    [
        {"role": "user", "content": "hi, what is going on?"}
    ],
    model=variant,
    stream=True,
    max_completion_tokens=10,
):
    print(token.choices[0].delta.content, end="")

Hello there! Not much is going on, just

In [13]:
pirate_features, relevance = client.features.search(
    "pirate",
    model=variant,
)
pirate_features

FeatureGroup([
   0: "Pirate-related language and themes",
   1: "Pirate characters and themes in fiction and role-playing games",
   2: "The model should roleplay as a pirate",
   3: "Mischievous behavior and troublemaking",
   4: "Mentions of rum, especially in pirate or cocktail contexts",
   5: "Willingness to do anything to achieve a goal, especially unethically",
   6: "Data transfer verbs in non-English languages",
   7: "Disguise or pretending to be someone else for infiltration or undercover purposes",
   8: "Entering or being inside an enclosed space",
   9: "Inclusion or being part of something"
])

In [14]:
relevance

[0.5869783759117126,
 0.5635083913803101,
 0.550840437412262,
 0.4005439579486847,
 0.3884296119213104,
 0.3563142716884613,
 0.35086721181869507,
 0.3506142497062683,
 0.35008418560028076,
 0.3474602699279785]

In [15]:
context = client.features.inspect(
    [
        {
            "role": "assistant",
            "content": "Hola, how are you? How is your day ?"
        },
        {
            "role": "user",
            "content": "Hola, how are you? How is your day ?"
        },
    ],
    model=variant,
)
matrix = context.matrix()
matrix

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 {31937: Feature("feature_31937"),
  21750: Feature("Conversation turn structure and role transitions"),
  34540: Feature("feature_34540"),
  21914: Feature("Beginning of a new conversation or input"),
  41765: Feature("Detection of new input or conversation start"),
  60918: Feature("End of message token in multi-turn conversations"),
  18: Feature("Contrastive conjunctions balancing difficulties with solutions"),
  15: Feature("The assistant needs clarification"),
  17: Feature("The model is providing troubleshooting steps or guidance"),
  1: Feature("Punctuation marks used for pacing and engagement in teaching contexts"),
  23361: Feature("Start of user input in conversation"),
  33849: Feature("The model's turn to speak in a conversation

In [34]:
context.top(k=5)._acts

[FeatureActivation(feature=Feature("Inquiring about someone's current state or condition"), activation=1.57890625),
 FeatureActivation(feature=Feature("Inquiring about someone's well-being or current state"), activation=1.352783203125),
 FeatureActivation(feature=Feature("Multilingual summary or conclusion request"), activation=0.68828125),
 FeatureActivation(feature=Feature("The model's turn to speak in a conversation"), activation=0.5986328125),
 FeatureActivation(feature=Feature("Portuguese event announcements and invitations"), activation=0.310546875)]

In [17]:
len(matrix[1])

105

In [18]:
for token in context.tokens:
    if not ("<|" in token.__str__() and "|>" in token.__str__()) :
        print(token)
        for act in token.inspect()._acts:
            print(act.feature.index_in_sae)
            print(act.activation)

Token("assistant")
23361
1.640625
33849
1.5390625
54609
1.0625
14765
0.8984375
34623
0.86328125
Token("

")
14765
2.953125
24720
0.55078125
7502
0.376953125
22244
0.32421875
52156
0.318359375
Token("Hola")
28127
3.25
20588
1.828125
43782
0.984375
11895
0.98046875
64587
0.9765625
Token(",")
50227
2.84375
33505
1.171875
64861
1.078125
65037
0.73828125
55311
0.71875
Token(" how")
50285
3.453125
8099
1.9296875
15357
1.9140625
19433
1.3203125
40064
1.21875
Token(" are")
43221
7.15625
63798
1.9296875
7878
0.37890625
6367
0.326171875
40103
0.2451171875
Token(" you")
48149
5.1875
47382
1.6015625
51233
1.1171875
59281
0.50390625
43221
0.421875
Token("?")
28031
2.03125
20533
1.21875
47382
1.0234375
2959
0.734375
61844
0.7265625
Token(" How")
19433
2.671875
50285
2.453125
15357
2.15625
50240
0.81640625
7878
0.734375
Token(" is")
7878
3.78125
9130
2.765625
27523
1.71875
43221
1.671875
6367
0.83203125
Token(" your")
7878
2.40625
64690
1.53125
42786
1.53125
48149
1.2734375
61893
1.171875
Token(" day

In [19]:
import networkx as nx
from itertools import combinations

G = nx.Graph()

for token in context.tokens:
    if not ("<|" in token.__str__() and "|>" in token.__str__()) :
        node_feature_ids = [act.feature.index_in_sae for act in token.inspect()._acts]
        feature_edges = [(u,v) for u, v in combinations(node_feature_ids, 2)]
        for edge in feature_edges:
            if G.has_edge(edge[0], edge[1]):
                G[edge[0]][edge[1]]["weight"]+=1
            else:
                G.add_edge(edge[0],edge[1], weight=1)



In [20]:
print(G.edges(data=True))

[(23361, 33849, {'weight': 1}), (23361, 54609, {'weight': 1}), (23361, 14765, {'weight': 1}), (23361, 34623, {'weight': 1}), (33849, 54609, {'weight': 1}), (33849, 14765, {'weight': 1}), (33849, 34623, {'weight': 1}), (54609, 14765, {'weight': 1}), (54609, 34623, {'weight': 1}), (14765, 34623, {'weight': 1}), (14765, 24720, {'weight': 1}), (14765, 7502, {'weight': 1}), (14765, 22244, {'weight': 1}), (14765, 52156, {'weight': 1}), (24720, 7502, {'weight': 1}), (24720, 22244, {'weight': 1}), (24720, 52156, {'weight': 1}), (7502, 22244, {'weight': 1}), (7502, 52156, {'weight': 1}), (7502, 53598, {'weight': 1}), (7502, 63745, {'weight': 1}), (7502, 4676, {'weight': 1}), (7502, 64861, {'weight': 1}), (22244, 52156, {'weight': 1}), (28127, 20588, {'weight': 2}), (28127, 43782, {'weight': 1}), (28127, 11895, {'weight': 1}), (28127, 64587, {'weight': 2}), (28127, 47378, {'weight': 1}), (28127, 3625, {'weight': 1}), (20588, 43782, {'weight': 1}), (20588, 11895, {'weight': 1}), (20588, 64587, {'

In [21]:
from ipysigma import Sigma



Sigma(G, node_color_gradient='Viridis', start_layout=10)

Sigma(nx.Graph with 76 nodes and 215 edges)

In [22]:
context = client.features.inspect(
    [
        {
            "role": "user",
            "content": "Hello, how are you? How is your day ?"
        },
    ],
    model=variant,
)
context.top(10)

FeatureActivations(
   0: (Feature("User's initial greeting to start a conversation"), 3.484375)
   1: (Feature("Greeting or inquiry using 'how' across languages"), 2.1354166666666665)
   2: (Feature("Inquiring about someone's well-being or status"), 2.0572916666666665)
   3: (Feature("Inquiring about someone's current state or condition"), 2.00146484375)
   4: (Feature("User initiating conversation with a greeting"), 1.59765625)
   5: (Feature("Inquiring about someone's well-being or current state"), 1.405029296875)
   6: (Feature("Informal conversation starters like 'Hi' or 'Hey'"), 0.7451171875)
   7: (Feature("The model's turn to speak in a conversation"), 0.5673828125)
   8: (Feature("Multilingual summary or conclusion request"), 0.46261160714285715)
   9: (Feature("The model's opening greeting and offer of help"), 0.4560546875)
)

In [23]:
context = client.features.inspect(
    [
        {
            "role": "user",
            "content": "Hola, how are you? How is your day ?"
        },
        {
            "role": "user",
            "content": "Hola, how are you? How is your day ?"
        },
        {
            "role": "user",
            "content": "Hola, how are you? How is your day ?"
        },
        {
            "role": "user",
            "content": "Hola, how are you? How is your day ?"
        },
                {
            "role": "user",
            "content": "Hola, how are you? How is your day ?"
        },
    ],
    model=variant,
)
top_10_feature = context.top(10)

In [24]:
context = client.features.inspect(
    [
        {
            "role": "user",
            "content": "Hola, how are you? How is your day ?"
        }
    ],
    model=variant,
)
context.top(10)

FeatureActivations(
   0: (Feature("Greeting or inquiry using 'how' across languages"), 2.1126302083333335)
   1: (Feature("Inquiring about someone's well-being or status"), 2.0234375)
   2: (Feature("Inquiring about someone's current state or condition"), 1.57890625)
   3: (Feature("Multilingual greetings and conversation starters"), 1.4150390625)
   4: (Feature("Inquiring about someone's well-being or current state"), 1.352783203125)
   5: (Feature("Spanish greetings and salutations"), 0.76171875)
   6: (Feature("End of model's response, user's turn to speak"), 0.6943359375)
   7: (Feature("Multilingual summary or conclusion request"), 0.68828125)
   8: (Feature("The model's turn to speak in a conversation"), 0.5986328125)
   9: (Feature("Portuguese event announcements and invitations"), 0.310546875)
)

In [25]:
import networkx as nx
from itertools import combinations

G = nx.Graph()

for token in context.tokens:
    if not ("<|" in token.__str__() and "|>" in token.__str__()) :
        node_feature_ids = [act.feature.index_in_sae for act in token.inspect()._acts]
        node_feature_activations = [act.activation for act in token.inspect()._acts]
        feature_edges = [(u,v) for u, v in combinations(node_feature_ids, 2)]
        for edge in feature_edges:
            if G.has_edge(edge[0], edge[1]):
                G[edge[0]][edge[1]]["weight"]+=1
            else:
                G.add_edge(edge[0],edge[1], weight=1)

In [26]:
G = nx.Graph()

node_feature_ids = ["A", "B", "C", "D"]
feature_edges = [(u,v) for u, v in combinations(node_feature_ids, 2)]
for edge in feature_edges:
    if G.has_edge(edge[0], edge[1]):
        G[edge[0]][edge[1]]["weight"]+=1
    else:
        G.add_edge(edge[0],edge[1], weight=1)

node_feature_ids = ["A", "B"]
feature_edges = [(u,v) for u, v in combinations(node_feature_ids, 2)]
for edge in feature_edges:
    if G.has_edge(edge[0], edge[1]):
        G[edge[0]][edge[1]]["weight"]+=1
    else:
        G.add_edge(edge[0],edge[1], weight=1)

node_feature_ids = ["A", "B","C"]
feature_edges = [(u,v) for u, v in combinations(node_feature_ids, 2)]
for edge in feature_edges:
    if G.has_edge(edge[0], edge[1]):
        G[edge[0]][edge[1]]["weight"]+=1
    else:
        G.add_edge(edge[0],edge[1], weight=1)

In [27]:
Sigma(G, node_color_gradient='Viridis', start_layout=10, edge_size= "weight")

Sigma(nx.Graph with 4 nodes and 6 edges)